<a href="https://colab.research.google.com/github/mahb97/Wake2vec/blob/main/token_injection_and_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ask and i shall deliver

REZZMAU5 WE1 | Tomorrowland 2024

In [1]:
!pip -q install transformers==4.43.3 accelerate peft==0.11.1 datasets umap-learn faiss-cpu matplotlib==3.8.4 sentencepiece

import os, re, math, random, json, unicodedata
from pathlib import Path
from collections import Counter
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
DTYPE = torch.bfloat16 if torch.cuda.is_available() else torch.float32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 8.2 MB/s eta 0:00:00
Device: cuda


In [3]:
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, torch_dtype=DTYPE, device_map="auto"
)
print("Base vocab size:", tok.vocab_size)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Base vocab size: 32000


In [4]:
# wake_lexicon.txt
lex_path = None
try:
    from google.colab import files
    uploaded = files.upload()
    lex_path = list(uploaded.keys())[0]
except Exception:
    pass

if lex_path is None:
    lex_path = "/content/wake_lexicon.txt"

raw = Path(lex_path).read_text(encoding="utf-8", errors="ignore").splitlines()

seen = set(); new_terms = []
for w in raw:
    t = unicodedata.normalize("NFC", w.strip())
    if not t:
        continue
    if t not in seen and t not in tok.get_vocab():
        seen.add(t); new_terms.append(t)

print(f"Loaded {len(new_terms)} NEW tokens (after dropping any already in vocab).")
print("Sample:", new_terms[:20])

Saving wake_lexicon-3.txt to wake_lexicon-3 (1).txt
Loaded 44500 NEW tokens (after dropping any already in vocab).
Sample: ['paùpulation', 'générations', 'introdùce', 'fainéants', 'tricarême', 'deathfête', 'grandmère', 'primamère', 'brofèsor', "andrée's", 'pailleté', 'ténèbres', "angèle's", 'choquées', 'traversé', 'arrivées', 'fraîches', 'willingdone', 'earwicker', 'lipoleums']


In [5]:
added = tok.add_tokens(new_terms, special_tokens=False)
if added != len(new_terms):
    print(f"Note: only {added}/{len(new_terms)} were newly added (collisions or tokenizer rules).")
model.resize_token_embeddings(len(tok))
print("New vocab size:", tok.vocab_size)

New vocab size: 32000


In [6]:
emb = model.get_input_embeddings()

def init_vec(term: str, noise_scale=0.02):
    with torch.no_grad():
        ids = tok(term, add_special_tokens=False)["input_ids"]
        if not ids:
            base = torch.randn_like(emb.weight[0:1])
        else:
            base = emb.weight[ids].mean(dim=0, keepdim=True)
        return (base + noise_scale * torch.randn_like(base)).squeeze(0)

with torch.no_grad():
    for t in new_terms:
        idx = tok.convert_tokens_to_ids(t)
        if idx != tok.unk_token_id:
            emb.weight[idx].copy_(init_vec(t))

print("Initialised", len(new_terms), "embedding rows.")

Initialised 44500 embedding rows.


In [7]:
for p in model.parameters():
    p.requires_grad = False

# Unfreeze input embeddings
model.get_input_embeddings().weight.requires_grad_(True)

# Call my daughter LoRA
USE_LORA = True
if USE_LORA:
    lora_cfg = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05,
        target_modules=["q_proj","k_proj","v_proj","o_proj"],  # TinyLlama on the mound
        bias="none", task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_cfg)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable params:", trainable)

Trainable params: 2252800


In [8]:
# FW
fw_path = None
try:
    from google.colab import files
    uploaded = files.upload()  # finnegans_wake.txt
    fw_path = list(uploaded.keys())[0]
except Exception:
    fw_path = "/content/finnegans_wake.txt"

fw = Path(fw_path).read_text(encoding="utf-8", errors="ignore")
fw = unicodedata.normalize("NFC", fw).replace("\r\n","\n").replace("\r","\n")

lines = [ln.strip() for ln in fw.split("\n") if ln.strip()]
lc_terms = {t.lower() for t in new_terms if len(t) <= 40}

hits = []
for ln in lines:
    lcln = ln.lower()
    if any(t in lcln for t in lc_terms):
        hits.append(ln)

# If too few matches top up with random lines
if len(hits) < 200:
    import random
    print("Few direct matches — topping up random lines.")
    hits += random.sample(lines, k=min(2000, len(lines)))

print("Training lines:", len(hits))

ds = Dataset.from_dict({"text": hits})

def tok_fn(batch):
    return tok(batch["text"], truncation=True, max_length=512)

ds_tok = ds.map(tok_fn, batched=True, remove_columns=["text"])
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

Saving FinnegansWake.txt to FinnegansWake.txt
Training lines: 21212


Map:   0%|          | 0/21212 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir="/content/Wake2vec_adapter",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-4,          # slightly higher: few params are moving
    num_train_epochs=1.5,         # quick pass; can raise to 2–3
    bf16=torch.cuda.is_available(),
    logging_steps=10,
    save_steps=500,
    save_total_limit=1,
    report_to=[]
)
trainer = Trainer(model=model, args=args, train_dataset=ds_tok, data_collator=collator)
trainer.train()
trainer.save_model("/content/Wake2vec_adapter/final")
tok.save_pretrained("/content/Wake2vec_adapter/final_tok")
print("Saved to /content/Wake2vec_adapter")


Step,Training Loss
10,7.039600
20,6.647400
30,6.512600
40,6.550500
50,6.425700
60,6.406900
70,6.367800
80,6.279100
90,6.402100
100,6.274900


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import umap
import matplotlib.pyplot as plt
import numpy as np

# Nearest neighbors among full vocab
def nearest_neighbors(terms, k=8):
    ids = [tok.convert_tokens_to_ids(t) for t in terms]
    W = emb.weight.detach().cpu().numpy()
    out = {}
    inv = {i:t for t,i in tok.get_vocab().items()}
    for term, idx in zip(terms, ids):
        if idx == tok.unk_token_id:
            continue
        v = W[idx:idx+1]
        sim = cosine_similarity(v, W)[0]
        top = np.argsort(-sim)[:k]
        out[term] = [inv.get(int(j), f"<{j}>") for j in top]
    return out

probe_new = new_terms[:30]
nn = nearest_neighbors(probe_new, k=8)
for k,v in nn.items():
    print(f"{k:>22} -> {v}")

# mix new tokens and some anchors
anchors = ["river","history","book","dream","night","language","think","place","irish","dublin"]
mix = list(dict.fromkeys((probe_new[:40] + anchors)))  # dedupe, preserve order
mix_ids = [tok.convert_tokens_to_ids(t) for t in mix if tok.convert_tokens_to_ids(t) != tok.unk_token_id]
mix_vecs = emb.weight[mix_ids].detach().cpu().numpy()

mapper = umap.UMAP(n_neighbors=8, min_dist=0.1, metric="cosine", random_state=SEED)
xy = mapper.fit_transform(mix_vecs)
plt.figure(figsize=(6,6))
plt.scatter(xy[:,0], xy[:,1])
inv = {i:t for t,i in tok.get_vocab().items()}
for pid, (x,y) in zip(mix_ids, xy):
    plt.text(x, y, inv.get(int(pid), "?"), fontsize=8)
plt.title("Wake2vec — local neighborhood map")
plt.show()

# Intrusion-style completions
def complete(prompt, max_new_tokens=40):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.9, top_p=0.92)
    return tok.decode(out[0], skip_special_tokens=True)

tests = [
    "By the river, I thought of",
    "At night, I dream of a",
    "In the story of this book,",
    "Explain gradient descent in the style of"
]
for p in tests:
    out = complete(p)
    print("\n===", p, "\n", out)


In [ ]:
save_dir = Path("/content/Wake2vec_adapter/minipack")
save_dir.mkdir(parents=True, exist_ok=True)

changed_ids = [tok.convert_tokens_to_ids(t) for t in new_terms if tok.convert_tokens_to_ids(t) != tok.unk_token_id]
emb_slice = emb.weight[changed_ids].detach().cpu().numpy()

np.save(save_dir / "new_token_ids.npy", np.array(changed_ids, dtype=np.int32))
np.save(save_dir / "new_token_vectors.npy", emb_slice)

(Path(save_dir / "README.txt").write_text(
    "Mini adapter: rows from the input embedding matrix for Wake2vec tokens.\n"
    "Apply by loading the base model, then assigning these rows by index\n"
    "into model.get_input_embeddings().weight.\n",
    encoding="utf-8"
))
print("Wrote minimal embedding rows to:", save_dir)
